In [1]:
# import libraries
import numpy as np # linear algebra, matrix multiplications
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from math import sqrt
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split

from keras import backend as K
import datetime

# for the architecture
from keras.models import Sequential
from keras.layers import Dense, Dropout, Lambda, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPool2D, AvgPool2D

# optimizer, data generator and learning rate reductor
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

from sklearn.metrics import confusion_matrix
import itertools

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import keras_metrics

Using TensorFlow backend.


In [8]:
def df_reshape(df, dim):
    return df.values.reshape(-1, dim, dim, 1)

def load_and_prepare_data(filename):
    data = pd.read_csv(filename, index_col=[0]).sample(frac=1).reset_index(drop=True)
    labels = data.pop('label').values

    dim = int(sqrt(data.shape[1]))

    # Normalize the data
    data = data / 255.0
    data = data.fillna(1.0)
    data = df_reshape(data, dim) # numpy.ndarray type

    # number of classes, in this case 2
    nclasses = labels.max() - labels.min() + 1
    labels = to_categorical(labels, num_classes = nclasses)

    # fix random seed for reproducibility
    seed = 2
    np.random.seed(seed)

    # percentage of xtrain which will be xtest
    split_pct = 0.2

    # Split the train and the validation set
    X_train, X_test, Y_train, Y_test = train_test_split(
        data, 
        labels, 
        test_size=split_pct,
        random_state=seed,
        shuffle=True,
        stratify=labels
    )

    return X_train, X_test, Y_train, Y_test

def build_model():
    model = Sequential()

    dim = 28
    nclasses = 2

    model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu', input_shape=(dim,dim,1)))
    model.add(Conv2D(filters=32, kernel_size=(5,5), padding='same', activation='relu',))
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(120, activation='relu'))
    model.add(Dense(84, activation='relu'))
    model.add(Dense(nclasses, activation='softmax'))
    
    return model

def train_model(model, epochs, batch_size):
    
    lr_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                 patience=3, 
                                 verbose=1, 
                                 factor=0.5, 
                                 min_lr=0.00001)
    
    datagen = ImageDataGenerator(
          featurewise_center=False,            # set input mean to 0 over the dataset
          samplewise_center=False,             # set each sample mean to 0
          featurewise_std_normalization=False, # divide inputs by std of the dataset
          samplewise_std_normalization=False,  # divide each input by its std
          zca_whitening=False,                 # apply ZCA whitening
          rotation_range=0,                   # randomly rotate images in the range (degrees, 0 to 180)
          zoom_range =0,                    # Randomly zoom image 
          width_shift_range=0,               # randomly shift images horizontally (fraction of total width)
          height_shift_range=0,              # randomly shift images vertically (fraction of total height)
          horizontal_flip=False,               # randomly flip images
          vertical_flip=False)                 # randomly flip images

    datagen.fit(X_train)

    model.compile(optimizer="adam", loss="mean_squared_error", metrics=["accuracy", 
                                                                        keras_metrics.precision(), 
                                                                        keras_metrics.recall(),
                                                                        keras_metrics.f1_score()])
    epochs = epochs
    batch_size = batch_size

    print('Start')
    print(datetime.datetime.now())
    history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=batch_size),
                              epochs=epochs, 
                              validation_data=(X_test, Y_test),
                              verbose=1, 
                              steps_per_epoch=X_train.shape[0] // batch_size, 
                              callbacks=[lr_reduction])

    print('End')
    print(datetime.datetime.now())
    
    return model

In [5]:
X_train, X_test, Y_train, Y_test = load_and_prepare_data('./obrazy_concat_3000.csv')

In [6]:
model = build_model()
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        832       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        25632     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6272)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 120)               752760    
_________________________________________________________________
dense_2 (Dense)              (None, 84)                10164     
__________

In [9]:
model = train_model(model, 15, 64)

Start
2019-05-13 13:12:33.389354
Epoch 1/15
75/75 [==============================] - 40s 532ms/step - loss: 0.2089 - acc: 0.6590 - precision: 0.6570 - recall: 0.6644 - f1_score: 0.6607 - val_loss: 0.1549 - val_acc: 0.8142 - val_precision: 0.9542 - val_recall: 0.6600 - val_f1_score: 0.7803
Epoch 2/15
75/75 [==============================] - 44s 583ms/step - loss: 0.0797 - acc: 0.9090 - precision: 0.9299 - recall: 0.8845 - f1_score: 0.9066 - val_loss: 0.0436 - val_acc: 0.9417 - val_precision: 0.9749 - val_recall: 0.9067 - val_f1_score: 0.9396
Epoch 3/15
75/75 [==============================] - 43s 574ms/step - loss: 0.0338 - acc: 0.9573 - precision: 0.9594 - recall: 0.9550 - f1_score: 0.9572 - val_loss: 0.0321 - val_acc: 0.9608 - val_precision: 0.9555 - val_recall: 0.9667 - val_f1_score: 0.9611
Epoch 4/15
75/75 [==============================] - 43s 569ms/step - loss: 0.0274 - acc: 0.9652 - precision: 0.9596 - recall: 0.9712 - f1_score: 0.9654 - val_loss: 0.0290 - val_acc: 0.9650 - val_p

In [10]:
model.save("nn_3000")

In [11]:
X_train, X_test, Y_train, Y_test = load_and_prepare_data('./obrazy_concat_2000.csv')
model = build_model()
model = train_model(model, 15, 64)
model.save("nn_2000")

Start
2019-05-13 13:25:47.759662
Epoch 1/15
50/50 [==============================] - 27s 532ms/step - loss: 0.2315 - acc: 0.6175 - precision: 0.6141 - recall: 0.6310 - f1_score: 0.6225 - val_loss: 0.1970 - val_acc: 0.6913 - val_precision: 0.6414 - val_recall: 0.8675 - val_f1_score: 0.7375
Epoch 2/15
50/50 [==============================] - 28s 563ms/step - loss: 0.1687 - acc: 0.7616 - precision: 0.7419 - recall: 0.8018 - f1_score: 0.7707 - val_loss: 0.1214 - val_acc: 0.8900 - val_precision: 0.9756 - val_recall: 0.8000 - val_f1_score: 0.8791
Epoch 3/15
50/50 [==============================] - 30s 601ms/step - loss: 0.0800 - acc: 0.9153 - precision: 0.9351 - recall: 0.8924 - f1_score: 0.9133 - val_loss: 0.0533 - val_acc: 0.9500 - val_precision: 0.9865 - val_recall: 0.9125 - val_f1_score: 0.9481
Epoch 4/15
50/50 [==============================] - 35s 699ms/step - loss: 0.0495 - acc: 0.9469 - precision: 0.9811 - recall: 0.9112 - f1_score: 0.9449 - val_loss: 0.0338 - val_acc: 0.9575 - val_p

In [12]:
X_train, X_test, Y_train, Y_test = load_and_prepare_data('./obrazy_concat_1000.csv')
model = build_model()
model = train_model(model, 15, 64)
model.save("nn_1000")

Start
2019-05-13 13:36:49.317117
Epoch 1/15
25/25 [==============================] - 13s 509ms/step - loss: 0.2276 - acc: 0.6394 - precision: 0.6316 - recall: 0.6687 - f1_score: 0.6497 - val_loss: 0.1822 - val_acc: 0.7175 - val_precision: 0.7164 - val_recall: 0.7200 - val_f1_score: 0.7182
Epoch 2/15
25/25 [==============================] - 13s 521ms/step - loss: 0.1873 - acc: 0.7175 - precision: 0.7122 - recall: 0.7300 - f1_score: 0.7210 - val_loss: 0.1716 - val_acc: 0.7250 - val_precision: 0.6800 - val_recall: 0.8500 - val_f1_score: 0.7556
Epoch 3/15
25/25 [==============================] - 14s 575ms/step - loss: 0.1616 - acc: 0.7619 - precision: 0.7416 - recall: 0.8037 - f1_score: 0.7714 - val_loss: 0.1575 - val_acc: 0.7450 - val_precision: 0.7753 - val_recall: 0.6900 - val_f1_score: 0.7302
Epoch 4/15
25/25 [==============================] - 15s 581ms/step - loss: 0.1329 - acc: 0.8287 - precision: 0.8231 - recall: 0.8375 - f1_score: 0.8302 - val_loss: 0.1145 - val_acc: 0.8800 - val_p

In [13]:
X_train, X_test, Y_train, Y_test = load_and_prepare_data('./obrazy_concat_500.csv')
model = build_model()
model = train_model(model, 15, 64)
model.save("nn_500")

Start
2019-05-13 13:41:34.309567
Epoch 1/15
12/12 [==============================] - 6s 531ms/step - loss: 0.2428 - acc: 0.5547 - precision: 0.5710 - recall: 0.4949 - f1_score: 0.5302 - val_loss: 0.2099 - val_acc: 0.7050 - val_precision: 0.8254 - val_recall: 0.5200 - val_f1_score: 0.6380
Epoch 2/15
12/12 [==============================] - 6s 475ms/step - loss: 0.2139 - acc: 0.6823 - precision: 0.6649 - recall: 0.7091 - f1_score: 0.6863 - val_loss: 0.2013 - val_acc: 0.7100 - val_precision: 0.8281 - val_recall: 0.5300 - val_f1_score: 0.6463
Epoch 3/15
12/12 [==============================] - 6s 490ms/step - loss: 0.1938 - acc: 0.7020 - precision: 0.7170 - recall: 0.6941 - f1_score: 0.7054 - val_loss: 0.1699 - val_acc: 0.7650 - val_precision: 0.7304 - val_recall: 0.8400 - val_f1_score: 0.7814
Epoch 4/15
12/12 [==============================] - 7s 596ms/step - loss: 0.1821 - acc: 0.7289 - precision: 0.6892 - recall: 0.7790 - f1_score: 0.7314 - val_loss: 0.1645 - val_acc: 0.7500 - val_preci

In [14]:
X_train, X_test, Y_train, Y_test = load_and_prepare_data('./obrazy_concat_100.csv')
model = build_model()
model = train_model(model, 15, 64)
model.save("nn_100")

Start
2019-05-13 13:44:08.743530
Epoch 1/15
2/2 [==============================] - 2s 875ms/step - loss: 0.2793 - acc: 0.3984 - precision: 0.4062 - recall: 0.4000 - f1_score: 0.4031 - val_loss: 0.2482 - val_acc: 0.5750 - val_precision: 1.0000 - val_recall: 0.1500 - val_f1_score: 0.2609
Epoch 2/15
2/2 [==============================] - 1s 475ms/step - loss: 0.2500 - acc: 0.5104 - precision: 0.6667 - recall: 0.0408 - f1_score: 0.0769 - val_loss: 0.2463 - val_acc: 0.5000 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_f1_score: 0.0000e+00
Epoch 3/15
2/2 [==============================] - 1s 403ms/step - loss: 0.2521 - acc: 0.4045 - precision: 0.3437 - recall: 0.2391 - f1_score: 0.2821 - val_loss: 0.2449 - val_acc: 0.5500 - val_precision: 1.0000 - val_recall: 0.1000 - val_f1_score: 0.1818
Epoch 4/15
2/2 [==============================] - 1s 504ms/step - loss: 0.2413 - acc: 0.5625 - precision: 1.0000 - recall: 0.0820 - f1_score: 0.1515 - val_loss: 0.2435 - val_acc: 0.5000 - val_p